# 전환이 일어나지 않은 유저(only view)에게 view_ratio 기준으로 추천

# 라이브러리 로드

In [1]:
!pip install implicit
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 1.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 2.1 MB/s 
     |████████████████████████████████| 1.6 MB 67.5 MB/s 


In [2]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit

from tqdm import tqdm

# 데이터 로드

In [3]:
try:
  path = 'C:/Users/User/Desktop/AIB_13/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')
except:
  path = '/content/drive/MyDrive/CP2/data/'
  df = pd.read_parquet(path + 'light_2019-Oct.parquet', engine='fastparquet')


In [4]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,-251657396,NaN,shiseido,35.790001,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,-780140327,appliances.environment.water_heater,aqua,33.200001,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,-1904213353,furniture.living_room.sofa,NaN,543.099976,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,1518338663,computers.notebook,lenovo,251.740005,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,-1769995873,electronics.smartphone,apple,1081.979980,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [5]:
data = df[['user_id','product_id','event_type']]

# 데이터 전처리

## view만 존재하는 유저 데이터 추출

1. cart, purchase 이력이 있는 유저들의 id값 받아오기
2. 위에서 얻은 cart, purchase 이력이 있는 유저들을 제외하기

In [6]:
drop_user_id = data.loc[data['event_type'] != 'view', 'user_id']

In [7]:
data = data.loc[~data['user_id'].isin(drop_user_id)].reset_index(drop=True)
data['event_type'].unique()

['view']
Categories (3, object): ['cart', 'purchase', 'view']

In [8]:
# event_type dtype 을 object로 변경

data['event_type'] = data['event_type'].astype('object')

## view 비율 구하기
- 해당 제품을 본 수 / 해당 유저가 전체 제품을 본 수

In [9]:
data.head()

,user_id,product_id,event_type
0,554748717,3900821,view
1,519107250,17200506,view
2,550050854,1307067,view
3,535871217,1004237,view
4,555447699,17300353,view


In [10]:
grouped = data.groupby(['user_id','product_id'])['event_type'].count()
grouped = grouped.reset_index()
grouped =grouped.rename(columns = {'event_type' : 'view_count'})

In [11]:
grouped

,user_id,product_id,view_count
0,33869381,7002639,1
1,64078358,10600284,1
2,183503497,22200103,1
3,184265397,6902133,2
4,184265397,6902303,2
...,...,...,...
15364922,566280663,1005127,2
15364923,566280676,13201002,1
15364924,566280697,2300307,1
15364925,566280780,15100003,1


In [12]:
total_event_type = data.groupby(['user_id'])['product_id'].count()
total_event_type = total_event_type.reset_index()
total_event_type = total_event_type.rename(columns={'product_id' : 'total_view'})

In [13]:
total_event_type

,user_id,total_view
0,33869381,1
1,64078358,1
2,183503497,1
3,184265397,6
4,195082191,1
...,...,...
2540827,566280663,2
2540828,566280676,1
2540829,566280697,1
2540830,566280780,1


In [14]:
data = grouped.merge(total_event_type, on='user_id')

In [15]:
data['view_ratio'] =  (data['view_count'] / data['total_view']) * 100
data.head()

,user_id,product_id,view_count,total_view,view_ratio
0,33869381,7002639,1,1,100.000000
1,64078358,10600284,1,1,100.000000
2,183503497,22200103,1,1,100.000000
3,184265397,6902133,2,6,33.333333
4,184265397,6902303,2,6,33.333333


# Item Table(product lookup) 테이블 만들기

In [16]:
product_lookup = df[['product_id','category_code','brand']].drop_duplicates('product_id').reset_index(drop=True).sort_values('product_id')
product_lookup.head()

,product_id,category_code,brand
151915,1000978,electronics.smartphone,NaN
8437,1001588,electronics.smartphone,meizu
85152,1001606,electronics.smartphone,apple
32556,1002042,electronics.smartphone,samsung
9400,1002062,electronics.smartphone,samsung


## Rating Matrix 만들기

In [17]:
num_user = data['user_id'].nunique()
num_item = data['product_id'].nunique()

num_user, num_item

(2540832, 159298)

In [18]:
users = list(np.sort(data['user_id'].unique()))
products = list(data['product_id'].unique())
ratio = list(data['view_ratio'])

rows = data['user_id'].astype('category').cat.codes
data['user_id_code'] = data['user_id'].astype('category').cat.codes

cols = data['product_id'].astype('category').cat.codes
data['product_id_code'] = data['product_id'].astype('category').cat.codes

len(users), len(products), len(ratio)

(2540832, 159298, 15364927)

In [19]:
user_item_matrix = sparse.csr_matrix((ratio, (rows, cols)), shape=(num_user, num_item))
user_item_matrix

<2540832x159298 sparse matrix of type '<class 'numpy.float64'>'
	with 15364927 stored elements in Compressed Sparse Row format>

# 추천 시스템 class 구현

In [20]:
from implicit.als import AlternatingLeastSquares
from sklearn import metrics
class MyALS():
  """
  implicit 라이브러리를 이용하여 필요한 기능을 담았습니다.

  Parameters
  ----------
  model : implicit 라이브러리의 AlternatingLeastSquares() 클래스로 생성된 인스턴스
  user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
  item_lookup : item(product)에 대한 정보를 담은 테이블
  data : User-Item Matrix를 만든 원본데이터

  """
  def __init__(self, model, user_item_matrix, item_lookup, data):

    self.model = model
    self.metrics_model = model
    self.user_item_matrix = user_item_matrix
    self.item_lookup = item_lookup
    self.data = data

  def fit(self, user_item_matrix):
    """
    행렬분해의 ALS를 이용하여 모델을 학습합니다.

    Parameters
    ----------
    user_item_matrix : 사용자가 정의한 User-Item Matrix(Sparse Matrix)
    """
    self.model.fit(user_item_matrix)
    
  def _user_id_2_code(self, user_id):
    """
    입력받은 user_id를 User_Item_Matrix에 있는 user_id_code로 바꾸어주는 함수

    Parameters
    ----------
    user_id : 유저 ID

    """
    user_id_code = self.data.loc[self.data['user_id'] == user_id, 'user_id_code'].unique()[0]
    return user_id_code

  def _product_id_2_code(self, product_id):
    """
    입력받은 product_id를 User-Item-Matrix에 있는 product_id_code로 바꾸어주는 함수

    Parameters
    ----------
    product_id : 상품 ID
    """
    product_id_code = self.data.loc[self.data['product_id'] == product_id, 'product_id_code'].unique()[0]
    return product_id_code

  def _code_2_product_id(self, product_id_code):
    """
    입력받은 product_id_code를 User-Item-Matrix에 있는 product_id로 바꾸어주는 함수

    Parameters
    ----------
    product_id_code : 상품 ID code    
    """
    product_id = self.data.loc[self.data['product_id_code'] == product_id_code, 'product_id'].unique()[0]
    return product_id

  def get_recom_product(self, user_id, n = 10):
    """  
    user_id에 맞는 product를 n개 만큼 추천하여 데이터프레임 형태로 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    n : 추천 받게 될 item의 수
    """ 

    # user_id_2_code 함수를 이용하여 유저의 ID를 user_id_code로 변환합니다
    user_id_code = self._user_id_2_code(user_id)
  
    # model의 recommend를 이용하여 추천받는 제품의 id를 추출합니다.
    # 이때 추천 받는 제품의 id는 product_id가 아니라 product_id_code 입니다.
    recommended = self.model.recommend(user_id_code, self.user_item_matrix[user_id_code], N=n)[0]
    #결과를 담을 리스트를 초기화 합니다.
    results = []
    # 추천 받은 id를 돌면서 item_lookup 테이블에서 해당 product의 정보를 찾아 결과에 담습니다.
    for product_id_code in recommended:
      
      recommended_product_id = self._code_2_product_id(product_id_code)
      result = self.item_lookup.loc[self.item_lookup['product_id'] == recommended_product_id]
      results.append(result)
      
    return pd.concat(results)
  
  def get_user_topN_product(self,user_id,column, n = 20):
    """
    유저가 특정 기준값이 높은 제품 N개를 반환

    Parameters
    ----------
    user_id : 유저 ID
    column : 값을 확인할 기준이 되는 컬럼
    n : 반환할 Item 수
    """
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 product_id를 추출
    product_ids = self.data.loc[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n]['product_id'].values
    #입력받은 user_id 를 기준으로 column이 높은 순으로 정렬하여 column을 추출
    product_values = self.data[self.data['user_id'] == user_id].sort_values(column, ascending=False)[:n][column].values

    results = []
    #item_lookup 테이블에서 id에 맞는 데이터프레임을 찾음
    for i in product_ids:
      result = self.item_lookup.loc[self.item_lookup['product_id'] == i]
      results.append(result)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame[column] = product_values

    return frame

  def get_explain(self, user_id, item_id, column):
    """
    사용자에게 제품이 추천된 이유를 반환하는 함수

    Parameters
    ----------
    user_id : 유저 ID
    item_id : Item(product) ID
    column : 확인할 컬럼
    """
    #입력받은 user_id, item_id를 user_id_code, product_id_code로 바꾸어줌
    user_id_code = self._user_id_2_code(user_id)
    product_id_code = self._product_id_2_code(item_id)

    #implicit라이브러리의 explain 함수를 사용하여 결과값을 반환
    total_score, top_contributions, user_weights = self.model.to_cpu().explain(user_id_code, self.user_item_matrix, product_id_code)

    results = []
    categorys = []
    brands = []
    scores = []
    # id에 해당하는 user_id, product_id, column, category, brand를 찾기
    for id_, score_ in top_contributions:
      product_id = self._code_2_product_id(id_)
      result = self.data.loc[(self.data['product_id'] == product_id) & (self.data['user_id'] == user_id)][['user_id','product_id',column]]
      category = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'category_code'].unique()[0]
      brand = self.item_lookup.loc[self.item_lookup['product_id'] == product_id, 'brand'].unique()[0]

      results.append(result)
      categorys.append(category)
      brands.append(brand)
      scores.append(score_)

    #결과를 확인하기 쉽게 데이터프레임으로 반환
    frame = pd.concat(results)
    frame['score'] = scores
    frame['category'] = categorys
    frame['brand'] = brands
    
    frame = frame[['user_id', 'product_id','category','brand',column, 'score']]
    return frame, total_score

  def _get_train_test(self,percentage=.2, seed=42):
    """
    score를 구하기 위하여 train, test 데이터를 나누어주는 함수
    파라미터로 들어오는 percentage만큼 train_set의 값을 0으로 만들어 줌
    test_set는 기존의 User-Item Matrix에서 0이 아닌값으로 모두 1로 만들어 줌

    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율
    seed : random seed

    """
    #원본 데이터를 test_set, train_set에 복사
    test_set = self.user_item_matrix.copy()
    train_set = self.user_item_matrix.copy()

    #relevant(선호 혹은 평가)여부를 확인하기 위하여 test_set에서 0이 아닌값을 1로 만들어 줌 
    test_set[test_set != 0] = 1

    #train_set에서 0이 아닌 x축, y축을 추출
    nonzero_idxs = train_set.nonzero()
    #x, y를 짝을지어 저장
    nonzero_pairs = list(zip(nonzero_idxs[0], nonzero_idxs[1]))

    #랜덤 시드를 적용
    random.seed(seed)
    #주어진 비율로 샘플을 추출
    n_samples = int(np.ceil(percentage * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, n_samples)

    
    user_idxs = [index[0] for index in samples]
    item_idxs = [index[1] for index in samples]

    #샘플에 해당 하는 값들을 평가한적이 없도록 보이기 위하여 0으로 감춤
    train_set[user_idxs, item_idxs] = 0
    
    train_set.eliminate_zeros()

    self.zero_user_idxs = user_idxs
    self.zero_item_idxs = item_idxs
    self.train_set = train_set
    self.test_set = test_set
    

  def get_score(self, percentage =.2, seed=42, k = 10, method='hit_at_k', n_samples = 10000):
    """
    train_set로 학습하고 test_set와 비교하여 method 파라미터를 이용하여 추천시스템의 성능을 평가하는 함수
    
    Parameters
    ----------
    percentage : 감추고 싶은 데이터의 비율, default = .2
    seed : random seed, default = 42
    k : 추천할 아이템의 수, default = 10
    method : 평가 지표 - hit_at_k, precision_at_k
    n_samples : 평가할 user의 수, default = 10000

    """
    # 입력받은 n_samples가 최대값이상 이면 최대값으로 적용
    max_num = self.data['user_id_code'].nunique()
    if max_num <= n_samples:
      n_samples = max_num

    # train, test로 데이터가 분리
    self._get_train_test(percentage, seed)
    # metrics_model이 학습
    self.metrics_model.fit(self.train_set)
    # 샘플 user를 랜덤으로 추출
    random_state = np.random.RandomState(seed)
    user_id_code_samples = random_state.choice(self.data['user_id_code'], n_samples)

    #method 방법에 따라 scores 값을 반환
    if method == 'hit_at_k':
      scores = self._hit_at_k(user_id_code_samples, k)
    elif method =='presicion_at_k':
      scores = self._precision_at_k(user_id_code_samples, k)
    return scores

  def _hit_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 relevant한것(존재)이 있다면 1, 아니면 0을 반환하여 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]

      results = []
      # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 만약 결과 리스트안에 1이 있다면 1을 입력, 아니면 0을 입력
      if 1 in results:
        scores.append(1)
      else:
        scores.append(0)
        #결과를 평균내어 반환
    return np.mean(scores)

  def _precision_at_k(self, user_id_code_samples, k):
    """
    k개의 추천 중 사용자가 relevant(선호 혹으 평가)한 아이템이 얼마나 존재하는지 측정
    추천을 받은 user 수 만큼 나누어 평균을 반환

    Parameters
    ----------
    user_id_code_samples : 샘플링한 유저의 ID 리스트
    k : 추천할 아이템의 수
    """
    scores = []
    #입력받은 user_id_code_samples를 돌면서
    for user_id_code_sample in tqdm(user_id_code_samples):
      #해당 유저에게 추천하는 아이템을 추출
      recommedation_ids = self.metrics_model.recommend(user_id_code_sample, self.train_set[user_id_code_sample], N=k)[0]
      results = []
       # 추천 받은 아이템이 유저가 선호(혹은 평가)했는지 확인
      for id_ in recommedation_ids:
        result = self.test_set[user_id_code_sample, id_]
        results.append(result)
      # 유저가 추천받은 아이템들을 얼마나 선호(혹은 평가)했는지 추출
      scores.append(np.mean(results))
      # 결과를 평균내어 반환
    return np.mean(scores)

# 추천하기

In [21]:
q1 = np.quantile(data['view_ratio'], 0.25)
q3 = np.quantile(data['view_ratio'], 0.75)
q1, q3

(2.666666666666667, 16.666666666666664)

In [22]:
data.loc[(data['view_ratio'] >= q1)&(data['view_ratio'] <= q3)].head()

,user_id,product_id,view_count,total_view,view_ratio,user_id_code,product_id_code
13,209714031,5500032,2,22,9.090909,8,19995
14,209714031,5500097,1,22,4.545455,8,20007
15,209714031,5500107,2,22,9.090909,8,20012
16,209714031,5500108,2,22,9.090909,8,20013
17,209714031,5500268,1,22,4.545455,8,20068


view_ratio가 2.6 ~ 16.6 사이에 있는 유저들 중에 한명을 골라 확인해보겠습니다.

In [23]:
model = MyALS(AlternatingLeastSquares(), user_item_matrix, product_lookup, data)

In [24]:
model.fit(user_item_matrix)

  0%|          | 0/15 [00:00<?, ?it/s]

먼저 이 유저의 view_ratio가 높은 제품을 확인해보겠습니다.

In [25]:
user = 209714031
column = 'view_ratio'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,view_ratio
17706,5500007,NaN,panasonic,18.181818
13292,26900028,sport.trainer,housefit,13.636364
11638,5500032,NaN,panasonic,9.090909
34678,5500107,NaN,braun,9.090909
46741,5500108,NaN,panasonic,9.090909
5638,44500029,NaN,omabelle,9.090909
34615,5500097,NaN,philips,4.545455
117782,5500268,NaN,panasonic,4.545455
40097,8700249,appliances.personal.hair_cutter,galaxy,4.545455
14124,8700258,appliances.personal.hair_cutter,remington,4.545455


여러 카테고리가 비어있지만 카테고리 종류 중 hair_cutter나  
브랜드로는 panasonic, braun, philips 와 같은 브랜드를 본 비율이 높은것을 유추 해볼때 면도기 종류를 검색해본것으로 추측합니다.

이 유저에게 모델이 어떤 제품을 추천했는지 확인해보겠습니다.

In [26]:
model.get_recom_product(user)

,product_id,category_code,brand
15472,44500004,NaN,omabelle
6094,44500031,NaN,omabelle
62982,44500015,NaN,omabelle
5463,44500023,NaN,omabelle
143620,51800038,NaN,NaN
143626,49500007,NaN,NaN
143624,51800034,NaN,NaN
79630,44500016,NaN,omabelle
54759,44500026,NaN,omabelle
6556,44500006,NaN,omabelle


유저가 본 브랜드 중 omabelle 브랜드의 제품들을 다수 추천한 것으로 보입니다.

맨 위에 위치한 44500004 id를 가진 제품을 어떤 이유로 추천했는지 확인해보겠습니다.

In [27]:
explain_frame, total_score = model.get_explain(user, 44500004, column)
explain_frame

/usr/local/lib/python3.7/dist-packages/implicit/utils.py:29: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  "OpenBLAS detected. Its highly recommend to set the environment variable "


,user_id,product_id,category,brand,view_ratio,score
23,209714031,44500029,NaN,omabelle,9.090909,0.214090
24,209714031,52000065,NaN,NaN,4.545455,0.065195
12,209714031,5500007,NaN,panasonic,18.181818,0.000340
21,209714031,8700583,appliances.personal.hair_cutter,braun,4.545455,0.000331
15,209714031,5500107,NaN,braun,9.090909,0.000117
13,209714031,5500032,NaN,panasonic,9.090909,0.000037
17,209714031,5500268,NaN,panasonic,4.545455,0.000024
14,209714031,5500097,NaN,philips,4.545455,0.000020
18,209714031,8700249,appliances.personal.hair_cutter,galaxy,4.545455,0.000003
16,209714031,5500108,NaN,panasonic,9.090909,-0.000011


다음은 512475445 유저에 관하여 확인해보겠습니다.

해당 유저는 우리 사이트에서 제품을 본 수가 가장 많은 유저입니다.

In [28]:
user = 512475445
column = 'view_ratio'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,view_ratio
868,4700419,auto.accessories.videoregister,sho-me,3.066165
763,4700478,auto.accessories.videoregister,sho-me,3.025820
4726,4700557,auto.accessories.videoregister,sho-me,2.837547
4864,4700590,auto.accessories.videoregister,sho-me,2.770307
14595,5800802,electronics.audio.subwoofer,kenwood,2.649274
2019,5700788,auto.accessories.player,kenwood,2.595481
2814,5701062,auto.accessories.player,pioneer,2.474449
4495,5701086,auto.accessories.player,pioneer,2.474449
487,5700384,auto.accessories.player,pioneer,1.707907
23871,6100194,auto.accessories.radar,NaN,1.654115


In [29]:
model.get_recom_product(user)

,product_id,category_code,brand
1786,5701128,auto.accessories.player,NaN
753,5701166,auto.accessories.player,NaN
2267,5801218,electronics.audio.subwoofer,NaN
4729,5701246,auto.accessories.player,NaN
690,4700630,auto.accessories.videoregister,NaN
367,5700518,auto.accessories.player,NaN
2345,5701247,auto.accessories.player,NaN
5682,5700282,auto.accessories.player,alpine
7023,5701020,auto.accessories.player,NaN
4405,5701087,auto.accessories.player,jvc


In [30]:
explain_frame, total_score = model.get_explain(user, 5701128, column)
explain_frame

,user_id,product_id,category,brand,view_ratio,score
243987,512475445,5701086,auto.accessories.player,pioneer,2.474449,0.137264
243965,512475445,5700788,auto.accessories.player,kenwood,2.595481,0.088587
243955,512475445,5700384,auto.accessories.player,pioneer,1.707907,0.083668
244025,512475445,5801482,electronics.audio.subwoofer,pioneer,1.546530,0.047156
243984,512475445,5701079,auto.accessories.player,jvc,1.344809,0.043363
243980,512475445,5701062,auto.accessories.player,pioneer,2.474449,0.042560
244026,512475445,5801483,electronics.audio.subwoofer,pioneer,1.613771,0.040344
243981,512475445,5701063,auto.accessories.player,pioneer,1.519634,0.038223
243944,512475445,4700478,auto.accessories.videoregister,sho-me,3.025820,0.031103
243959,512475445,5700576,auto.accessories.player,pioneer,0.739645,0.028587


In [31]:
user = 519607186
column = 'view_ratio'

model.get_user_topN_product(user, column)

,product_id,category_code,brand,view_ratio
5969,1004887,electronics.smartphone,oppo,73.563218
1166,1005159,electronics.smartphone,xiaomi,1.149425
50,1002544,electronics.smartphone,apple,1.149425
171,15100367,NaN,NaN,1.149425
4187,14701391,furniture.living_room.cabinet,NaN,1.149425
1259,14701558,furniture.living_room.cabinet,brw,1.149425
19848,14701705,furniture.living_room.cabinet,brw,0.766284
40247,14700394,furniture.living_room.cabinet,NaN,0.766284
52673,14700738,furniture.living_room.cabinet,brw,0.766284
2286,15100147,NaN,lider,0.766284


In [32]:
model.get_recom_product(user)

,product_id,category_code,brand
141,1004838,electronics.smartphone,oppo
1113,1004886,electronics.smartphone,oppo
585,1004839,electronics.smartphone,oppo
1055,1004961,electronics.smartphone,oppo
2392,1004990,electronics.smartphone,oppo
120205,1005205,electronics.smartphone,NaN
524,1005021,electronics.smartphone,oppo
141303,1005238,electronics.smartphone,NaN
460,1004902,electronics.smartphone,oppo
6382,1004578,electronics.smartphone,oppo


In [33]:
explain_frame, total_score = model.get_explain(user, 1004838, column)
explain_frame

,user_id,product_id,category,brand,view_ratio,score
5085523,519607186,1004887,electronics.smartphone,oppo,73.563218,1.036344
5085522,519607186,1004856,electronics.smartphone,samsung,0.383142,0.014931
5085515,519607186,1002544,electronics.smartphone,apple,1.149425,0.013507
5085530,519607186,1005159,electronics.smartphone,xiaomi,1.149425,0.007178
5085521,519607186,1004794,electronics.smartphone,xiaomi,0.383142,0.006077
5085524,519607186,1005008,electronics.smartphone,xiaomi,0.383142,0.002963
5085533,519607186,1005239,electronics.smartphone,NaN,0.383142,0.001799
5085528,519607186,1005115,electronics.smartphone,apple,0.383142,0.001531
5085566,519607186,17200570,furniture.living_room.sofa,NaN,0.383142,0.001365
5085532,519607186,1005217,electronics.smartphone,NaN,0.383142,0.000641


# 성능 평가하기

In [34]:
scores = model.get_score(method='hit_at_k')
print()
print('hit@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:13<00:00, 760.25it/s]


hit@k(10) :  0.35


In [35]:
scores = model.get_score(method='presicion_at_k')
print()
print('precision@k(10) : ',round(scores, 2))

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 10000/10000 [00:13<00:00, 757.77it/s]


precision@k(10) :  0.05
